
# Project: E-commerce Product Inventory API (FastAPI + MongoDB)

This notebook walks you step-by-step through building and **calling** a simple E-commerce Product Inventory REST API.

### What you will learn
- REST API design for a real use case
- Product & inventory modeling
- MongoDB integration
- Calling APIs using `requests`
- Understanding stock reduction logic

This project is intentionally **minimal and beginner-friendly**.



## 1. Project Scenario (Business View)

We are building a backend for a simple e-commerce system.

### Requirements
- Admin can add products
- Products have inventory (stock)
- Users can view products
- Users can buy products
- Inventory reduces after purchase
- Purchase fails if stock is insufficient



## 2. API Design (REST)

| Action | Method | Endpoint |
|------|------|---------|
| Create product | POST | /products |
| List products | GET | /products |
| Get product | GET | /products/{id} |
| Update inventory | PATCH | /products/{id}/inventory |
| Purchase product | POST | /products/{id}/purchase |



## 3. Prerequisites

Before running API calls, ensure:

1. MongoDB is running locally  
`mongodb://localhost:27017`

2. FastAPI server is running  
```bash
uvicorn app.main_project:app --reload
```

3. Install required packages  
```bash
pip install requests
```


## 4. Base URL for API Calls

In [4]:

BASE_URL = "http://127.0.0.1:8000"


## 5. Create a Product (Admin)

In [5]:

import requests

payload = {
    "name": "Wireless Mouse",
    "description": "2.4GHz ergonomic mouse",
    "price": 799,
    "quantity": 50
}

response = requests.post(f"{BASE_URL}/products", json=payload)
response.status_code, response.json()


(201,
 {'name': 'Wireless Mouse',
  'description': '2.4GHz ergonomic mouse',
  'price': 799,
  'quantity': 50,
  'id': '6978638dcec88517a0ad2b84'})

## 6. List All Products

In [37]:

response = requests.get(f"{BASE_URL}/products")
response.status_code, response.json()


(200,
 [{'name': 'Wireless Mouse',
   'description': '2.4GHz ergonomic mouse',
   'price': 799,
   'quantity': 93,
   'id': '69734cf05b5b2b0d8b7e3138'}])


## 7. Get Product by ID

Copy a product ID from the previous response.


In [28]:

# PRODUCT_ID = "<PASTE_PRODUCT_ID_HERE>"
PRODUCT_ID = "69734cf05b5b2b0d8b7e3138"

response = requests.get(f"{BASE_URL}/products/{PRODUCT_ID}")
response.status_code, response.json()


(200,
 {'name': 'Wireless Mouse',
  'description': '2.4GHz ergonomic mouse',
  'price': 799,
  'quantity': 50,
  'id': '69734cf05b5b2b0d8b7e3138'})

## 8. Update Inventory (Admin)

In [29]:

payload = {
    "quantity": 100
}

response = requests.patch(
    f"{BASE_URL}/products/{PRODUCT_ID}/inventory",
    json=payload
)
response.status_code, response.json()


(200, {'message': 'Inventory updated', 'quantity': 100})

## 9. Purchase Product (Core Business Logic)

In [36]:

payload = {
    "quantity": 2
}

response = requests.post(
    f"{BASE_URL}/products/{PRODUCT_ID}/purchase",
    json=payload
)
response.status_code, response.json()


(200, {'message': 'Purchase successful', 'remaining_quantity': 96})

## 10. Purchase Failure Example (Insufficient Stock)

In [35]:

payload = {
    "quantity": 1000
}

response = requests.post(
    f"{BASE_URL}/products/{PRODUCT_ID}/purchase",
    json=payload
)
response.status_code, response.json()


(400, {'detail': 'Insufficient stock'})


## 11. What You Learned

- REST API resource design
- HTTP methods & status codes
- MongoDB-backed inventory
- Atomic stock reduction logic
- How frontend / clients call APIs

This project mirrors **real backend interviews & production systems**.



## 12. Next Improvements (Optional)

- Add Pydantic models
- Add MongoDB indexes
- Use `$inc` for atomic inventory updates
- Add order history collection
- Add authentication



## 13. Inspecting Data in MongoDB (Mongo Shell)

After calling the APIs, it’s important to **verify what is actually stored in MongoDB**.
This helps students understand how API actions translate into database changes.

We assume MongoDB is running locally.



### 13.1 Open Mongo Shell

In a new terminal window, run:

```bash
mongosh
```

If MongoDB is running correctly, you will see a Mongo shell prompt.



### 13.2 Switch to the Database

Our API uses the `ecommerce` database.


In [ ]:

// Mongo shell command
use ecommerce



### 13.3 List Collections

Check what collections exist in the database.


In [ ]:

// Mongo shell command
show collections



You should see a collection named `products`.



### 13.4 View All Products

Fetch all documents from the `products` collection.


In [ ]:

// Mongo shell command
db.products.find().pretty()



This output should match what you see from the `GET /products` API.



### 13.5 Find a Product by ID

Copy the `_id` value from the output above.


In [ ]:

// Mongo shell command
db.products.findOne({ _id: ObjectId("PASTE_OBJECT_ID_HERE") })



### 13.6 Verify Inventory Reduction After Purchase

After calling the **purchase API**, re-run:


In [ ]:

// Mongo shell command
db.products.find().pretty()



You should notice:
- `quantity` has reduced
- No new document is created (update in-place)

This confirms the **business logic is working correctly**.



### 13.7 Clean Up (Optional)

To delete all products (for fresh testing):


In [ ]:

// Mongo shell command
db.products.deleteMany({})



⚠️ Use delete operations carefully in real systems.



## 14. Key Takeaway (Very Important)

- APIs are just a layer on top of the database
- Always verify backend logic by inspecting the DB
- This habit helps debug production issues faster

Professional backend engineers **always check the DB**, not just API responses.
